In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [ ]:
import cv2
import numpy as np
from PIL import Image
import horizon.transforms as T

ir16 = cv2.imread(
    "/mnt/fiftyoneDB/Database/Image_Data/Thermal_Images_16Bit/Trip_310_Seq_153/23714953_r.png",
    cv2.IMREAD_UNCHANGED,
)
transform = T.horizon_augment_ir16bit(imgsz=640, im_compression_prob=0.9)
im1 = (transform(image=ir16)["image"].numpy().transpose(1, 2, 0) * 255).astype(np.uint8)

transform = T.horizon_base_ir16bit(imgsz=640)
im2 = (transform(image=ir16)["image"].numpy().transpose(1, 2, 0) * 255).astype(np.uint8)

rgb = cv2.imread(
    "/mnt/fiftyoneDB/Database/Image_Data/RGB_Images/Trip_310_Seq_153/23714953.jpg",
    cv2.IMREAD_UNCHANGED
)
transform = T.horizon_augment_rgb(imgsz=640, im_compression_prob=0.9)
im3 = (transform(image=rgb)["image"].numpy().transpose(1, 2, 0) * 255).astype(np.uint8)

transform = T.horizon_base_rgb(imgsz=640)
im4 = (transform(image=rgb)["image"].numpy().transpose(1, 2, 0) * 255).astype(np.uint8)

# stack in 2x2 grid
im = np.vstack([np.hstack([im1, im2]), np.hstack([im3, im4])])
Image.fromarray(im).reduce(2)
